https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5

In [19]:
import torch
import random
import torchaudio
from torchaudio import transforms
import os
from torch.utils.data import DataLoader, Dataset
from AudioUtil import AudioUtil

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
  def __init__(self, data_path):
    self.data_path = str(data_path)
    self.files = [f for f in os.listdir(data_path) if f.endswith('.wav')]
    self.duration = 2000
    self.sr = 44100
    self.channel = 2
    self.shift_pct = 0.4
            
  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.files)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path
    audio_file = self.files[idx]
    # Get the Class ID
    class_id = 0 if audio_file.split('_')[0] == 'noise' else 1

    aud = AudioUtil.open(os.path.join(self.data_path, audio_file))
    # Some sounds have a higher sample rate, or fewer channels compared to the
    # majority. So make all sounds have the same number of channels and same 
    # sample rate. Unless the sample rate is the same, the pad_trunc will still
    # result in arrays of different lengths, even though the sound duration is
    # the same.
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)

    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    return aug_sgram, class_id

In [20]:
train_data = SoundDS('data/train')
test_data = SoundDS('data/test')
valid_data = SoundDS('data/validation')

train_dl = DataLoader(train_data, batch_size=16, shuffle=True)
test_dl = DataLoader(test_data, batch_size=16, shuffle=True)
valid_dl = DataLoader(valid_data, batch_size=16, shuffle=True)

In [33]:
import torch.nn.functional as F
from torch.nn import init
import torch.nn as nn
from AudioClassifier import AudioClassifier

# Create the model and put it on the GPU if available
myModel = AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cuda', index=0)

In [22]:
print(torch.__version__)

2.3.0+cu118


In [34]:
import ipywidgets as widgets
print(widgets.IntSlider())

IntSlider(value=0)

In [42]:
# ----------------------------
# Training Loop
# ----------------------------
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.003)
  # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
  #                                               steps_per_epoch=int(len(train_dl)),
  #                                               epochs=num_epochs,
  #                                               anneal_strategy='linear')

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = data[0].to(device), data[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

        if i % 10 == 0:    # print every 10 mini-batches
           print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
    
    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')
  
num_epochs=16   # Just for demo, adjust this higher.
training(myModel, train_dl, num_epochs)

Epoch: 1/16, Loss: 0.16, Accuracy: 0.94
Epoch: 2/16, Loss: 0.11, Accuracy: 0.96
Epoch: 3/16, Loss: 0.14, Accuracy: 0.94
Epoch: 4/16, Loss: 0.14, Accuracy: 0.94
Epoch: 5/16, Loss: 0.15, Accuracy: 0.94
Epoch: 6/16, Loss: 0.12, Accuracy: 0.94
Epoch: 7/16, Loss: 0.13, Accuracy: 0.95
Epoch: 8/16, Loss: 0.12, Accuracy: 0.95
Epoch: 9/16, Loss: 0.15, Accuracy: 0.94
Epoch: 10/16, Loss: 0.12, Accuracy: 0.95
Epoch: 11/16, Loss: 0.15, Accuracy: 0.94
Epoch: 12/16, Loss: 0.11, Accuracy: 0.97
Epoch: 13/16, Loss: 0.10, Accuracy: 0.96
Epoch: 14/16, Loss: 0.11, Accuracy: 0.96
Epoch: 15/16, Loss: 0.13, Accuracy: 0.95
Epoch: 16/16, Loss: 0.12, Accuracy: 0.94
Finished Training


In [43]:
def inference(model, val_dl):
    correct_prediction = 0
    total_prediction = 0
    class_names = {0: 'Noise', 1: 'Knock'}  # Mapping class indices to class names

    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs, 1)
            # Print the predicted class names, using the key itself as the default value if not found in the dictionary
            predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction]
            actual_classes = [class_names.get(l.item(), l.item()) for l in labels]
            print(f'Predicted:\t{predicted_classes}\nActual:\t\t{actual_classes}')
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction / total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

# Assuming myModel and valid_dl are defined
inference(myModel, valid_dl)

Predicted:	['Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock']
Actual:		['Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Knock']
Predicted:	['Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Noise']
Actual:		['Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock']
Predicted:	['Knock', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock']
Actual:		['Knock', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock', 'Knock', 'Knock', 'Noise', 'Knock']
Predicted:	['Noise', 'Knock', 'Knock', 'Knock', 'Knock', 'Knock', 'Kno

In [27]:
#saving the model
torch.save(myModel.state_dict(), 'data/models/V8_model_state_dict.pth')
torch.save(myModel, 'data/models/V8_model_full.pth')

In [29]:
model = AudioClassifier()
model.load_state_dict(torch.load('data/models/V8_model_state_dict.pth'))
model = model.to(device)
single_ds = SoundDS('data/single')
single_dl = DataLoader(single_ds, shuffle=True)

inference(model, single_dl)

Predicted:	['Knock']
Actual:		['Knock']
Accuracy: 1.00, Total items: 1
